In [1]:
print("Hello")

Hello


In [2]:
from summary_non_py import get_summary_non_py
import pathlib
from utils import llm

In [3]:
paths = ["/workspace/code_qa_llm/test_repo/doc/layernorm/layernorm.md",
         "/workspace/code_qa_llm/test_repo/scripts/run_gpt2_124M.sh",
         ] 

In [4]:
res=[]
for path in paths:
        ans = get_summary_non_py(path)
        res.append(ans)
res

[[{'name': 'LayerNorm',
   'type': 'class',
   'summary': 'This class implements the forward and backward passes of Layer Normalization manually using simpler PyTorch operations. It includes static methods to perform the forward pass (normalizing input activations) and the backward pass (computing gradients).',
   'path': '/workspace/code_qa_llm/test_repo/doc/layernorm/layernorm.md',
   'code': 'import torch\neps = 1e-5\n\nclass LayerNorm:\n\n    @staticmethod\n    def forward(x, w, b):\n        # x is the input activations, of shape B,T,C\n        # w are the weights, of shape C\n        # b are the biases, of shape C\n        B, T, C = x.size()\n        # calculate the mean\n        mean = x.sum(-1, keepdim=True) / C # B,T,1\n        # calculate the variance\n        xshift = x - mean # B,T,C\n        var = (xshift**2).sum(-1, keepdim=True) / C # B,T,1\n        # calculate the inverse standard deviation: **0.5 is sqrt, **-0.5 is 1/sqrt\n        rstd = (var + eps) ** -0.5 # B,T,1\n   

In [4]:
ans

[{'name': 'main',
  'type': 'function',
  'summary': 'This function serves as the entry point for profiling the GPT-2 training using CUDA. It initializes the multi-GPU configuration, builds the GPT-2 model from a checkpoint, sets the batch size and sequence length, allocates memory for input and output data, overrides the number of layers, and performs a training step including forward pass, zeroing gradients, backward pass, and updating the model. Finally, it synchronizes CUDA devices and frees resources.',
  'path': 'test_repo/profile_gpt2.cu',
  'code': 'int main(int argc, char *argv[]) {\n        multi_gpu_config = multi_gpu_config_init(&argc, &argv);\n        common_start(true, true);\n\n        // build the GPT-2 model from a checkpoint\n        GPT2 model;\n        gpt2_build_from_checkpoint(&model, "gpt2_124M_bf16.bin");\n\n        int B = 24; // if program OOMs decrease this number, e.g. all the way down to 4 or etc\n        int T = 1024; // if even that OOMs move on to this o

In [6]:
from utils import file_read

In [7]:
summ = file_read("/workspace/code_qa_llm/sos/summaries.json")

In [8]:
summ

[{'name': 'run.py',
  'summary': "\nThe provided code initializes and runs a Flask web application. Firstly, it imports the `create_app` function from the `app` module and the `putModelScheduler` from `app.tasks.putModel`. The Flask app is created by calling `create_app()`. A route `/status` is defined which, when accessed via a GET request, returns the response 'Running!'. In the main execution block, the `putModelScheduler` is started, which seems to initiate a scheduler task. Finally, the Flask app runs on host `0.0.0.0` and port `5000` with debugging enabled.\n",
  'type': 'file',
  'code': 'from app import create_app\nfrom app.tasks.putModel import putModelScheduler\n\napp = create_app()\n\n@app.route(\'/status\', methods=[\'GET\'])\ndef status():\n    return \'Running!\'\n\nif __name__ == "__main__":\n    putModelScheduler.start()\n    app.run(host="0.0.0.0",port=5000, debug=True)',
  'path': '/workspace/groundcrew/smart-orchestration-service-master/smart-orchestration-service-ma

In [9]:
summary = ""
for element in summ:
    name = element['name']
    summary = element['summary']
    type = element['type']
    try :
        dependent_functions = element['dependent_functions']
    except Exception as e :
        dependent_functions = "null"
    path = element['path']
    summary += f"The name of the file is {name}. Its type is {type}. Its path is {path}. Its dependant functions are {dependent_functions}. Its summary is {summary}. "
    

In [10]:
summary

'\nThe code defines a Smart Orchestration Service for managing multiple Stable Diffusion (SD) models hosted on different servers. Below are the supported API endpoints:\n\n1. **Server-Mappings Endpoints:**\n   - **Register Server (`POST /server-mappings/register`):** Registers a server hosting an SD model with parameters `model_name` (string, required), `server_endpoint` (List<string>, required), and optional `redundancy_count` (int).\n   - **De-Register Server (`POST /server-mappings/de-register`):** De-registers a server with parameters `model_name` (string, required), `server_endpoint` (List<string>, required), and optionally `is_delete_key_required` (bool) and `deletion_count` (int).\n   - **Update Server (`POST /server-mappings/update`):** Updates a server address with parameters `model_name` (string, required), `old_server_endpoint` (string, required), and `new_server_endpoint` (string, required).\n   - **Get All Server Mappings (`GET /server-mappings/get-all`):** Retrieves all s

In [11]:
from create_summary import _get_token_length
num = _get_token_length (summary)
num

791

In [12]:
prompt=f"The summaries of all the files are : {summary}"
system_prompt="""You are a good code summarizer. Given summaries of all file in a repository , your task is to create an overall summary using all these summaries. This summary will be used for question answering about the code and code improvements , so build the summary accordingly.
    Generate your summary between <summary>...</summary> tag."""
summary=llm(prompt,system_prompt)
summary=summary.split('<summary>')[-1].split('</summary>')[0]



In [13]:
summary

'\nThe repository contains a Smart Orchestration Service designed to manage multiple Stable Diffusion (SD) models hosted on different servers. This service provides several API endpoints for efficient server management and monitoring:\n\n1. **Server-Mappings Endpoints:**\n   - **Register Server (`POST /server-mappings/register`):** Allows the registration of a new server hosting an SD model. Requires `model_name` and `server_endpoint` parameters, with an optional `redundancy_count`.\n   - **De-Register Server (`POST /server-mappings/de-register`):** Enables de-registration of an existing server with `model_name` and `server_endpoint` parameters. It also supports optional `is_delete_key_required` and `deletion_count` parameters.\n   - **Update Server (`POST /server-mappings/update`):** Facilitates updating the server address by specifying `model_name`, `old_server_endpoint`, and `new_server_endpoint`.\n   - **Get All Server Mappings (`GET /server-mappings/get-all`):** Retrieves all serv